In [11]:
import pandas as pd 
import cv2                 
import numpy as np         
import os                  
from random import shuffle
from tqdm import tqdm  
import scipy
import skimage
from skimage.transform import resize
#print(os.listdir("chest_xray"))

In [12]:
TRAIN_DIR = "train/"
TEST_DIR =  "test/"
VAL_DIR = "val/"

In [13]:
def get_label(Dir):
    for nextdir in os.listdir(Dir):
        if not nextdir.startswith('.'):
            if nextdir in ['NORMAL']:
                label = 0
            elif nextdir in ['PNEUMONIA']:
                label = 1
            else:
                label = 2
    return nextdir, label

In [14]:
def preprocessing_data(Dir):
    X = []
    y = []
    
    for nextdir in os.listdir(Dir):
        nextdir, label = get_label(Dir)
        temp = Dir + nextdir
        
        for image_filename in tqdm(os.listdir(temp)):
            path = os.path.join(temp + '/' , image_filename)
            img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = skimage.transform.resize(img, (150, 150, 3))
                img = np.asarray(img)
                X.append(img)
                y.append(label)
            
    X = np.asarray(X)
    y = np.asarray(y)
    
    return X,y

In [15]:
def get_data(Dir):
    X = []
    y = []
    for nextDir in os.listdir(Dir):
        if not nextDir.startswith('.'):
            if nextDir in ['NORMAL']:
                label = 0
            elif nextDir in ['PNEUMONIA']:
                label = 1
            else:
                label = 2
                
            temp = Dir + nextDir
                
            for file in tqdm(os.listdir(temp)):
                img = cv2.imread(temp + '/' + file)
                if img is not None:
                    img = skimage.transform.resize(img, (150, 150, 3))
                    #img_file = scipy.misc.imresize(arr=img_file, size=(150, 150, 3))
                    img = np.asarray(img)
                    X.append(img)
                    y.append(label)
                    
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [16]:
X_train, y_train = get_data(TRAIN_DIR)

  0%|          | 0/3877 [00:00<?, ?it/s]/Users/omkar/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/omkar/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
100%|██████████| 1343/1343 [02:36<00:00,  8.15it/s]


In [17]:
X_test , y_test = get_data(TEST_DIR)


100%|██████████| 234/234 [00:23<00:00, 10.04it/s]


In [18]:
print(X_train.shape,'\n',X_test.shape)

(5216, 150, 150, 3) 
 (624, 150, 150, 3)


In [19]:
print(y_train.shape,'\n',y_test.shape)

(5216,) 
 (624,)


In [20]:

from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [21]:

print(y_train.shape,'\n',y_test.shape)

(5216, 2) 
 (624, 2)


In [22]:
Pimages = os.listdir(TRAIN_DIR + "PNEUMONIA")
Nimages = os.listdir(TRAIN_DIR + "NORMAL")

In [23]:

from keras.callbacks import ReduceLROnPlateau , ModelCheckpoint
lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=1, verbose=1)

/Users/omkar/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [24]:
filepath="weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [25]:
from keras.models import Sequential
from keras.layers import Dense , Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , RMSprop
from keras.layers import Conv2D , BatchNormalization
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [26]:
X_train=X_train.reshape(5216,3,150,150)
X_test=X_test.reshape(624,3,150,150)

In [23]:

def swish_activation(x):
    return (K.sigmoid(x) * x)

model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', padding="same", input_shape=(3,150,150)))
model.add(Conv2D(16, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=(3,150,150)))
model.add(Conv2D(32, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(96, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(96, (3, 3), padding="valid", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(128, (3, 3), padding="valid", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64, activation=swish_activation))
model.add(Dropout(0.4))


model.add(Dense(2 , activation='sigmoid'))


model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(lr=0.00005),
                  metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 150, 150)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 150, 150)      2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 75, 75)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 75, 75)        4640      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 75, 75)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxP

In [35]:

from keras.models import Sequential
from keras.layers import Dense , Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , RMSprop
from keras.layers import Conv2D , BatchNormalization
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

model = Sequential()


model.add(Conv2D(filters=96, input_shape=(3,150,150), kernel_size=(9,9), strides=(2,2), padding="valid"))
model.add(Activation("relu"))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))


model.add(Conv2D(filters=256, kernel_size=(9,9), strides=(1,1), padding="valid"))
model.add(Activation("relu"))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))


model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid"))
model.add(Activation("relu"))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid"))
model.add(Activation("relu"))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="valid"))
model.add(Activation("relu"))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

model.add(Flatten())

# 1st Fully Connected Layer
model.add(Dense(4096 , activation="relu"))

# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Fully Connected Layer
model.add(Dense(4096 , activation="relu"))

# Add Dropout
model.add(Dropout(0.4))

# 3rd Fully Connected Layer
model.add(Dense(1000,activation="relu"))

# Add Dropout
model.add(Dropout(0.4))

model.add(Dense(2,activation="softmax"))


model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(lr=0.00005),
                  metrics=['accuracy'])
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 96, 71, 71)        23424     
_________________________________________________________________
activation_62 (Activation)   (None, 96, 71, 71)        0         
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 96, 35, 35)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 256, 27, 27)       1990912   
_________________________________________________________________
activation_63 (Activation)   (None, 256, 27, 27)       0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 256, 13, 13)       0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 384, 11, 11)       885120    
__________

In [36]:
batch_size = 256
epochs = 6




In [37]:

history = model.fit(X_train, y_train, validation_data = (X_test , y_test) ,callbacks=[lr_reduce,checkpoint] ,
          epochs=epochs)

Instructions for updating:
Use tf.cast instead.
Train on 5216 samples, validate on 624 samples
Epoch 1/6
5216/5216 [==============================] - 810s 155ms/step - loss: 0.4959 - acc: 0.7897 - val_loss: 0.6818 - val_acc: 0.7019

Epoch 00001: val_acc improved from -inf to 0.70192, saving model to weights.hdf5
Epoch 2/6
5216/5216 [==============================] - 879s 168ms/step - loss: 0.2802 - acc: 0.8911 - val_loss: 0.7493 - val_acc: 0.7276

Epoch 00002: val_acc improved from 0.70192 to 0.72756, saving model to weights.hdf5
Epoch 3/6
5216/5216 [==============================] - 841s 161ms/step - loss: 0.2386 - acc: 0.9087 - val_loss: 0.5285 - val_acc: 0.7644

Epoch 00003: val_acc improved from 0.72756 to 0.76442, saving model to weights.hdf5
Epoch 4/6
5216/5216 [==============================] - 773s 148ms/step - loss: 0.2053 - acc: 0.9179 - val_loss: 0.8595 - val_acc: 0.7035

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 00004: val_acc di

In [38]:
from sklearn.metrics import confusion_matrix
pred = model.predict(X_test)
pred = np.argmax(pred,axis = 1) 
y_true = np.argmax(y_test,axis = 1)

In [57]:
pred = model.predict(X_test)


In [58]:
pred

array([[3.5492392e-06, 9.9999642e-01],
       [2.5091898e-02, 9.7490805e-01],
       [2.5305671e-03, 9.9746943e-01],
       ...,
       [3.6205485e-02, 9.6379447e-01],
       [1.2806589e-03, 9.9871933e-01],
       [1.9707705e-03, 9.9802917e-01]], dtype=float32)

In [39]:
CM = confusion_matrix(y_true, pred)

#fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(5, 5))
#plt.show()

In [40]:
CM

array([[ 90, 144],
       [  7, 383]])

In [119]:
#recall

90/(90+15)

90.61378205128206

In [45]:
def get_val_data(Dir):
    X = []
    y =[]
    for nextDir in os.listdir(Dir):
        if not nextDir.startswith('.'):
            if nextDir in ['NORMAL']:
                label = 0
            elif nextDir in ['PNEUMONIA']:
                label = 1
            else:
                label = 2
                
            temp = Dir + nextDir
                
            for file in tqdm(os.listdir(temp)):
                img = cv2.imread(temp + '/' + file)
                if img is not None:
                    img = skimage.transform.resize(img, (150, 150, 3))
                    #img_file = scipy.misc.imresize(arr=img_file, size=(150, 150, 3))
                    img = np.asarray(img)
                    X.append(img)
                    y.append(label)
                    
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [67]:
X_val, y_val =get_val_data(VAL_DIR)

100%|██████████| 10/10 [00:00<00:00, 12.58it/s]


In [69]:
X_val=X_val.reshape(17,3,150,150)


In [70]:
prediction = model.predict(X_val)


In [75]:
prediction_val = np.argmax(prediction,axis = 1) 


In [76]:
CM = confusion_matrix(y_val, prediction_val)


In [78]:
CM

array([[6, 2],
       [0, 9]])

In [115]:
test_image = cv2.imread('val/NORMAL/NORMAL2-IM-1438-0001.jpeg')

In [116]:
test_image = skimage.transform.resize(test_image, (150, 150, 3))
test_image = np.asarray(test_image)
test_image=test_image.reshape(1,3,150,150)

In [117]:
prediction = model.predict(test_image)

In [118]:
prediction

array([[0.6048417 , 0.39515832]], dtype=float32)